In [19]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill

def is_date_or_digits(value):
  '''
  Check if the value is a date or a string of digits.
  '''
  value = str(value).strip()
  if value.isdigit():
    return True
  try:
    pd.to_datetime(value, errors='raise')
    return True
  except:
    return False

def preprocess_and_highlight(file_path,sheets):
  # Load workbook
  wb = load_workbook(file_path)
  yellow_fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")

  for sheet_name in sheets:
    df = pd.read_excel(file_path, sheet_name=sheet_name)
    df = df.dropna(subset=['Process Step'])
    df = df[df['Process Step'].astype(str).str.strip() != '']
    df = df.reset_index(drop=True)

    # Identify rows to hightlight
    highlight_indices = []
    for core in df['Core'].dropna().unique():
      core_mask = df['Core'] == core
      core_df = df[core_mask]
      process_steps = core_df['Process Step'].astype(str).str.strip()

      has_date_like = process_steps.apply(is_date_or_digits).any()

      if has_date_like:
        highlight_mask = core_mask & process_steps.apply(is_date_or_digits)
      else:
        highlight_mask = (
            core_mask &
            process_steps.str.contains('cad', case=False, na=False) & 
            core_df['Bearing Air'].str.contains('off', case=False, na=False) &
            core_df['Stroke Loc'].str.contains('bdc', case=False, na=False)
        )
      highlight_indices.extend(df[highlight_mask].index.tolist())

    ws = wb[sheet_name]
    ws.delete_rows(2, ws.max_row)

    for row in df.itertuples(index=False): 
      ws.append(row)
    for i in highlight_indices:
      for col in range(1, len(df.columns) + 1): 
        ws.cell(row=i+2, column=col).fill = yellow_fill

  wb.save(file_path)
  print("Preprocessing and Highlighting Completed")
      
if __name__ == "__main__":
  file_path = "Data.xlsx"
  sheets_to_process = ['J Core', 'K Core', 'M Core']
  preprocess_and_highlight(file_path,sheets_to_process)

Preprocessing and Highlighting Completed
